# Load in extracted features

In [80]:
# open the every file in a directory and print the first line of each file
import os
import sys
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import BayesianRidge
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder


In [96]:
# load in the features
features_train = np.array([])
for file in os.listdir('features_train'):
    with open('features_train/' + file) as f:
        df = pd.read_csv(f)
        features_train = np.column_stack((features_train, df.values)) if features_train.size else df.values

# load the labels
train = pd.read_csv('train.csv')
y_train = train['Genre']
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
# normalize the data
features_train = preprocessing.scale(features_train)

# Load in test Data


In [106]:
features_test = np.array([])
for file in os.listdir('features_test'):
    with open('features_test/' + file) as f:
        df = pd.read_csv(f)
        features_test = np.column_stack((features_test, df.values)) if features_test.size else df.values

# scale the test data
features_test = preprocessing.scale(features_test)


# Logistic Regression

In [107]:

logreg = LogisticRegression(max_iter=150)
logreg.fit(features_train, y_train)
predictions = logreg.predict(features_train)
# Calculate the accuracy
accuracy = accuracy_score(y_train, predictions)
print("Logistic Regression accuracy:", accuracy)


Logistic Regression accuracy: 0.8072590738423029


# Predict with Logistic Regression and save to CSV

In [109]:

# run the model on the test data
predictions = logreg.predict(features_test)
predictions = label_encoder.inverse_transform(predictions)
# save the predictions to a csv file
df = pd.DataFrame({'ID': os.listdir('test'), 'genre': predictions})
df.to_csv('predictions.csv', index=False)

# Neural Network

In [116]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(37, activation='relu', input_shape = (37,)))

model.add(Dense(37, activation='relu'))

model.add(Dense(37, activation='softmax'))

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(features_train, y_train, epochs=50, batch_size=20)

loss, accuracy = model.evaluate(features_train, y_train)
print('Neural Network accuracy:', accuracy)

Epoch 1/50


40/40 [==============================] - 0s 1ms/step - loss: 3.2872 - accuracy: 0.1289 
Epoch 2/50
40/40 [==============================] - 0s 974us/step - loss: 2.3155 - accuracy: 0.3579
Epoch 3/50
40/40 [==============================] - 0s 863us/step - loss: 1.7109 - accuracy: 0.4355
Epoch 4/50
40/40 [==============================] - 0s 876us/step - loss: 1.4421 - accuracy: 0.5144
Epoch 5/50
40/40 [==============================] - 0s 988us/step - loss: 1.2871 - accuracy: 0.5569
Epoch 6/50
40/40 [==============================] - 0s 888us/step - loss: 1.1848 - accuracy: 0.6058
Epoch 7/50
40/40 [==============================] - 0s 873us/step - loss: 1.1031 - accuracy: 0.6245
Epoch 8/50
40/40 [==============================] - 0s 894us/step - loss: 1.0505 - accuracy: 0.6446
Epoch 9/50
40/40 [==============================] - 0s 926us/step - loss: 0.9918 - accuracy: 0.6583
Epoch 10/50
40/40 [==============================] - 0s 977us/step - loss: 0.9463 - accuracy: 0.6596
Epoch 11/50

# Predict with Neural Network and save to CSV

In [117]:
predictions = model.predict(features_test)
predictions = np.argmax(predictions, axis=1)
predictions = label_encoder.inverse_transform(predictions)

print(predictions)

df = pd.DataFrame({'ID': os.listdir('test'), 'genre': predictions})
df.to_csv('predictions.csv', index=False)

7/7 [==============================] - 0s 0s/step


['hiphop' 'pop' 'country' 'metal' 'disco' 'rock' 'hiphop' 'classical'
 'blues' 'blues' 'country' 'metal' 'hiphop' 'disco' 'blues' 'blues'
 'metal' 'disco' 'pop' 'hiphop' 'jazz' 'hiphop' 'classical' 'hiphop'
 'rock' 'classical' 'country' 'disco' 'rock' 'disco' 'jazz' 'classical'
 'jazz' 'jazz' 'disco' 'pop' 'hiphop' 'jazz' 'jazz' 'country' 'pop'
 'metal' 'country' 'pop' 'jazz' 'disco' 'country' 'pop' 'country' 'disco'
 'disco' 'hiphop' 'reggae' 'pop' 'hiphop' 'reggae' 'pop' 'rock' 'hiphop'
 'country' 'classical' 'country' 'country' 'country' 'hiphop' 'metal'
 'pop' 'jazz' 'disco' 'metal' 'rock' 'hiphop' 'classical' 'metal'
 'country' 'reggae' 'disco' 'jazz' 'rock' 'disco' 'pop' 'disco' 'reggae'
 'country' 'classical' 'rock' 'classical' 'disco' 'blues' 'hiphop'
 'reggae' 'jazz' 'country' 'metal' 'blues' 'classical' 'disco' 'country'
 'disco' 'disco' 'blues' 'country' 'pop' 'metal' 'classical' 'pop' 'disco'
 'country' 'disco' 'jazz' 'rock' 'country' 'disco' 'classical' 'country'
 'reggae'